# UE5 Multi-Agent x LLM Reasoners

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
import time
import threading
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
from dotenv import load_dotenv
from A2A.agent import A2Agent
from UE.unrealcv_a2a import UnrealCvA2A
from UE.unrealcv_basic import UnrealCV
# from ue_agent.agent import UEAgent
# from ue_agent.plan_agent import PLAgent
from llm.openai_model import UEOpenAIModel
# from ue_agent.base import ActionBuffer
from Prompt.prompt import USER_PROMPT
from Config.config import Config

c:\Users\Shadow\miniconda3\envs\citynav\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !pip install python-dotenv

In [2]:
load_dotenv()

False

In [3]:
unrealcv_client_actor = UnrealCvA2A(port=9000, ip='127.0.0.1', resolution=(1280, 960))

INFO:__init__:230:Got connection confirm: b'connected to gym_citynav'


=>Info: using ip-port socket


In [4]:
name = f'GEN_DELIVERY_MAN_0'
path = Config.DELIVERY_MAN_MODEL_PATH
unrealcv_client_actor.spawn_bp_asset(path, name)

In [5]:
model = UEOpenAIModel("gpt-4o")

In [7]:
# user_prompt = USER_PROMPT.format(waypoint="(4,7)")

In [6]:
if __name__ == "__main__":


    Agent_Human = A2Agent(
        model=model,
        unrealcv_client=unrealcv_client_actor,
        name=name,
        temperature=1.5,
        max_history_step=5,
        camera_id=1,
        observation_viewmode='lit',
        # action_buffer=action_buffer
    )
    # Agent_Dog_1 = A2Agent(
    #     model=model,
    #     unrealcv_client=unrealcv_client_actor,
    #     name="BP_SpotRobot_C_1",
    #     camera_id=3,
    #     temperature=1.5,
    #     max_history_step=5,
    #     observation_viewmode='lit',
    #     # action_buffer=action_buffer
    # )
    # Agent_Dog_2 = A2Agent(
    #     model=model,
    #     unrealcv_client=unrealcv_client_actor,
    #     name="BP_SpotRobot_C_2",
    #     camera_id=2,
    #     temperature=1.5,
    #     max_history_step=5,
    #     observation_viewmode='lit',
    #     # action_buffer=action_buffer
    # )

    Agent_Human.reset()
    # Agent_Dog_1.reset()
    # Agent_Dog_2.reset()

    


2025-04-10 12:36:08,305 - A2A.agent - INFO - Current position: Vector(x=22.92, y=0.0)


In [ ]:
# Agent_Human.get_possible_next_waypoints()

[Vector(x=18300.0, y=1700.0), Vector(x=1700.0, y=-1700.0)]

In [7]:
Agent_Human.parse()

2025-04-10 12:37:14,900 - A2A.agent - INFO - Current position: Vector(x=22.92, y=0.0), Next waypoint: Vector(x=5000.0, y=1700.0), Direction: Vector(x=1.0, y=0.0)
2025-04-10 12:37:14,902 - A2A.agent - INFO - Aligning direction
2025-04-10 12:37:14,904 - A2A.agent - INFO - Angle to waypoint: 18.86199661546547
2025-04-10 12:37:14,905 - A2A.agent - INFO - Angle: 18.86199661546547, Turn direction: right
2025-04-10 12:37:14,954 - A2A.agent - INFO - Current position: Vector(x=22.92, y=0.0)
2025-04-10 12:37:14,958 - A2A.agent - INFO - Aligning direction
2025-04-10 12:37:14,962 - A2A.agent - INFO - Angle to waypoint: 17.86163409688865
2025-04-10 12:37:14,973 - A2A.agent - INFO - Angle: 17.86163409688865, Turn direction: right
2025-04-10 12:37:15,025 - A2A.agent - INFO - Current position: Vector(x=22.92, y=0.0)
2025-04-10 12:37:15,027 - A2A.agent - INFO - Aligning direction
2025-04-10 12:37:15,029 - A2A.agent - INFO - Angle to waypoint: 15.866777694429558
2025-04-10 12:37:15,031 - A2A.agent - INF

Response: {"actions":[0,0],"waypoints":[{"x":5000.0,"y":1700.0},{"x":1700.0,"y":-1700.0}]}
Next waypoint: Vector(x=5000.0, y=1700.0)


2025-04-10 12:37:15,079 - A2A.agent - INFO - Angle to waypoint: 14.85002280677411
2025-04-10 12:37:15,081 - A2A.agent - INFO - Angle: 14.85002280677411, Turn direction: right
2025-04-10 12:37:15,121 - A2A.agent - INFO - Current position: Vector(x=22.92, y=0.0)
2025-04-10 12:37:15,125 - A2A.agent - INFO - Aligning direction
2025-04-10 12:37:15,127 - A2A.agent - INFO - Angle to waypoint: 13.856162022382076
2025-04-10 12:37:15,128 - A2A.agent - INFO - Angle: 13.856162022382076, Turn direction: right
2025-04-10 12:37:15,165 - A2A.agent - INFO - Current position: Vector(x=22.92, y=0.0)
2025-04-10 12:37:15,168 - A2A.agent - INFO - Aligning direction
2025-04-10 12:37:15,170 - A2A.agent - INFO - Angle to waypoint: 12.864327848600016
2025-04-10 12:37:15,172 - A2A.agent - INFO - Angle: 12.864327848600016, Turn direction: right
2025-04-10 12:37:15,216 - A2A.agent - INFO - Current position: Vector(x=22.92, y=0.0)
2025-04-10 12:37:15,218 - A2A.agent - INFO - Aligning direction
2025-04-10 12:37:15,2

Next waypoint: Vector(x=1700.0, y=-1700.0)


2025-04-10 12:38:10,539 - A2A.agent - INFO - Current position: Vector(x=4843.649, y=1627.675)
2025-04-10 12:38:10,542 - A2A.agent - INFO - Aligning direction
2025-04-10 12:38:10,544 - A2A.agent - INFO - Angle to waypoint: 122.37104249386879
2025-04-10 12:38:10,545 - A2A.agent - INFO - Angle: 122.37104249386879, Turn direction: left
2025-04-10 12:38:10,591 - A2A.agent - INFO - Current position: Vector(x=4843.649, y=1627.675)
2025-04-10 12:38:10,593 - A2A.agent - INFO - Aligning direction
2025-04-10 12:38:10,595 - A2A.agent - INFO - Angle to waypoint: 115.37240398098554
2025-04-10 12:38:10,596 - A2A.agent - INFO - Angle: 115.37240398098554, Turn direction: left
2025-04-10 12:38:10,643 - A2A.agent - INFO - Current position: Vector(x=4843.649, y=1627.675)
2025-04-10 12:38:10,645 - A2A.agent - INFO - Aligning direction
2025-04-10 12:38:10,646 - A2A.agent - INFO - Angle to waypoint: 108.37288802984085
2025-04-10 12:38:10,648 - A2A.agent - INFO - Angle: 108.37288802984085, Turn direction: lef

In [11]:
unrealcv_client_actor.d_pick_up(name, None)

In [ ]:
Agent_Human.walk_arrive_at_waypoint()

True

In [ ]:
Agent_Human.next_waypoint

Vector(x=4.0, y=7.0)

In [ ]:
# take_action(Agent_Human, observation_space, description_space)

In [ ]:
# image = Agent_Dog_1.client.read_image(0, "depth", mode="direct")
# image